In [1]:
# ==========================================
# CELL 3: GLOBAL INFERENCE & SUBMISSION (FINAL PLATINUM)
# ==========================================
import joblib
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
import polars as pl
import os
import glob
import gc
import shutil
from tqdm.auto import tqdm
import warnings

warnings.filterwarnings('ignore')

# --- CẤU HÌNH ---
MODEL_PATH = '/kaggle/input/model-testdata/models' 
if not os.path.exists(MODEL_PATH):
    print(f"⚠️ Đường dẫn {MODEL_PATH} không tồn tại!")
    print("Các thư mục trong /kaggle/input/model-testdata là:")
    try:
        print(os.listdir('/kaggle/input/model-testdata'))
    except:
        print("Không đọc được thư mục cha.")
else:
    print(f"✅ Đường dẫn model OK: {MODEL_PATH}")
    print(f"   Tìm thấy {len(glob.glob(f'{MODEL_PATH}/*.pkl'))} file .pkl")

TEST_DATA_PATH = '/kaggle/input/MABe-mouse-behavior-detection'
SUBMISSION_FILE = 'submission.csv'

# --- FEATURE COLUMNS (PLATINUM - Phải khớp 100% với Cell 1 & 2) ---
FEATURE_COLS = [
    'distance', 'velocity_m1', 'velocity_m2', 'accel_m1', 'accel_m2',
    'curvature_m1', 'curvature_m2', 'turn_rate_m1', 'turn_rate_m2', 'speed_ratio_m1',
    'nose1_to_tail2', 'nose1_to_nose2', 'facing_angle_m1',
    'elongation_m1', 'elongation_m2', 'spine_alignment',
    'vel_m1_std_10', 'vel_m2_std_10', 'grooming_score_m1',
    'dist_mean_10', 'vel_m1_mean_10', 'vel_m2_mean_10', 
    'facing_mean_10', 'curvature_m1_mean_10'
]

# --- HÀM XỬ LÝ DỮ LIỆU (Được nhúng trực tiếp để độc lập với Cell 1) ---
def calculate_features_polars(df_pl):
    df_pl = df_pl.sort("frame")
    
    # --- GIAI ĐOẠN 1: CÁC ĐẠI LƯỢNG CƠ BẢN (BASICS) ---
    exprs_basic = [
        # Distance: Body Center to Body Center
        (((pl.col("mouse1_body_center_x") - pl.col("mouse2_body_center_x"))**2 + 
          (pl.col("mouse1_body_center_y") - pl.col("mouse2_body_center_y"))**2).sqrt()).fill_null(0).alias("distance"),
        
        # Velocity Magnitude (Speed)
        (((pl.col("mouse1_body_center_x").diff().fill_null(0))**2 + 
          (pl.col("mouse1_body_center_y").diff().fill_null(0))**2).sqrt()).alias("velocity_m1"),
        (((pl.col("mouse2_body_center_x").diff().fill_null(0))**2 + 
          (pl.col("mouse2_body_center_y").diff().fill_null(0))**2).sqrt()).alias("velocity_m2"),
          
        # Social Distances
        (((pl.col("mouse1_nose_x") - pl.col("mouse2_tail_base_x"))**2 + 
          (pl.col("mouse1_nose_y") - pl.col("mouse2_tail_base_y"))**2).sqrt()).fill_null(0).alias("nose1_to_tail2"),
        (((pl.col("mouse1_nose_x") - pl.col("mouse2_nose_x"))**2 + 
          (pl.col("mouse1_nose_y") - pl.col("mouse2_nose_y"))**2).sqrt()).fill_null(0).alias("nose1_to_nose2"),
    ]
    df_pl = df_pl.with_columns(exprs_basic)

    # --- GIAI ĐOẠN 2: TÍNH TOÁN VECTOR & HELPER COLUMNS ---
    # Tạo các cột vector để dùng cho tính toán phức tạp phía sau
    df_pl = df_pl.with_columns([
        # Vector Vận tốc (Velocity Vector)
        pl.col("mouse1_body_center_x").diff().fill_null(0).alias("vx1"),
        pl.col("mouse1_body_center_y").diff().fill_null(0).alias("vy1"),
        pl.col("mouse2_body_center_x").diff().fill_null(0).alias("vx2"),
        pl.col("mouse2_body_center_y").diff().fill_null(0).alias("vy2"),
        
        # Vector Xương Sống (Spine Vector): Từ Mũi -> Đuôi (Dùng thay cho Neck để an toàn dữ liệu thiếu)
        (pl.col("mouse1_tail_base_x") - pl.col("mouse1_nose_x")).alias("spine1_x"),
        (pl.col("mouse1_tail_base_y") - pl.col("mouse1_nose_y")).alias("spine1_y"),
        (pl.col("mouse2_tail_base_x") - pl.col("mouse2_nose_x")).alias("spine2_x"),
        (pl.col("mouse2_tail_base_y") - pl.col("mouse2_nose_y")).alias("spine2_y"),

        # Vector Mũi tương đối (Relative Nose): Mũi so với Trọng tâm (Để bắt Grooming)
        (pl.col("mouse1_nose_x") - pl.col("mouse1_body_center_x")).alias("nose1_rel_x"),
        (pl.col("mouse1_nose_y") - pl.col("mouse1_body_center_y")).alias("nose1_rel_y"),
    ])
    
    # Tính Vector Gia tốc (Acceleration Vector) từ Vận tốc
    df_pl = df_pl.with_columns([
        pl.col("vx1").diff().fill_null(0).alias("ax1"),
        pl.col("vy1").diff().fill_null(0).alias("ay1"),
        pl.col("vx2").diff().fill_null(0).alias("ax2"),
        pl.col("vy2").diff().fill_null(0).alias("ay2")
    ])
    
    # --- GIAI ĐOẠN 3: PLATINUM FEATURES (LOGIC PHỨC TẠP) ---
    
    # 1. Spine Alignment & Elongation (Độ thẳng hàng & Độ dài cơ thể)
    norm_spine1 = (pl.col("spine1_x")**2 + pl.col("spine1_y")**2).sqrt()
    norm_spine2 = (pl.col("spine2_x")**2 + pl.col("spine2_y")**2).sqrt()
    dot_spine = pl.col("spine1_x") * pl.col("spine2_x") + pl.col("spine1_y") * pl.col("spine2_y")
    
    # 2. Micro-Motion (Grooming Indicator)
    # Tính vận tốc lắc lư của mũi
    nose_speed_rel = ((pl.col("nose1_rel_x").diff().fill_null(0))**2 + 
                      (pl.col("nose1_rel_y").diff().fill_null(0))**2).sqrt()

    # 3. Facing Angle
    v1_x = pl.col('mouse1_nose_x') - pl.col('mouse1_body_center_x')
    v1_y = pl.col('mouse1_nose_y') - pl.col('mouse1_body_center_y')
    v12_x = pl.col('mouse2_body_center_x') - pl.col('mouse1_body_center_x')
    v12_y = pl.col('mouse2_body_center_y') - pl.col('mouse1_body_center_y')
    dot_face = v1_x * v12_x + v1_y * v12_y
    norm_face1 = (v1_x**2 + v1_y**2).sqrt()
    norm_face12 = (v12_x**2 + v12_y**2).sqrt()

    # 4. Turn Rate (Góc di chuyển)
    move_angle1 = pl.arctan2(pl.col("vy1"), pl.col("vx1"))
    move_angle2 = pl.arctan2(pl.col("vy2"), pl.col("vx2"))

    # --- TỔNG HỢP FEATURES ---
    df_pl = df_pl.with_columns([
        # Acceleration Magnitude
        pl.col("velocity_m1").diff().fill_null(0).alias("accel_m1"),
        pl.col("velocity_m2").diff().fill_null(0).alias("accel_m2"),
        
        # [NEW] Spine Alignment: -1 (Đối đầu), 1 (Song song cùng chiều - Mount)
        (dot_spine / (norm_spine1 * norm_spine2 + 1e-6)).fill_nan(0).alias("spine_alignment"),
        
        # [NEW] Elongation: Độ dài cơ thể (Attack dướn người vs Huddle co người)
        norm_spine1.alias("elongation_m1"),
        norm_spine2.alias("elongation_m2"),
        
        # [NEW] Grooming Score: Mũi di chuyển nhiều / Thân di chuyển ít
        (nose_speed_rel / (pl.col("velocity_m1") + 0.5)).fill_nan(0).alias("grooming_score_m1"),

        # Facing Angle
        (dot_face / (norm_face1 * norm_face12 + 1e-6)).fill_nan(0).fill_null(0).alias('facing_angle_m1'),
        
        # Curvature
        ( (pl.col("vx1")*pl.col("ay1") - pl.col("vy1")*pl.col("ax1")).abs() / 
          ((pl.col("vx1")**2 + pl.col("vy1")**2 + 1e-6)**1.5) ).fill_nan(0).alias("curvature_m1"),
        ( (pl.col("vx2")*pl.col("ay2") - pl.col("vy2")*pl.col("ax2")).abs() / 
          ((pl.col("vx2")**2 + pl.col("vy2")**2 + 1e-6)**1.5) ).fill_nan(0).alias("curvature_m2"),

        # Turn Rate & Speed Ratio
        move_angle1.diff().abs().fill_null(0).alias("turn_rate_m1"),
        move_angle2.diff().abs().fill_null(0).alias("turn_rate_m2"),
        (pl.col("velocity_m1") / (pl.col("velocity_m1").rolling_mean(30).fill_null(0) + 1e-6)).alias("speed_ratio_m1")
    ])
    
    # --- GIAI ĐOẠN 4: ROLLING WINDOWS (CONTEXT & JITTER) ---
    roll_exprs = [
        # Context (Mean 10 frames)
        pl.col("distance").rolling_mean(10).fill_null(0).alias("dist_mean_10"),
        pl.col("velocity_m1").rolling_mean(10).fill_null(0).alias("vel_m1_mean_10"),
        pl.col("velocity_m2").rolling_mean(10).fill_null(0).alias("vel_m2_mean_10"),
        pl.col("facing_angle_m1").rolling_mean(10).fill_null(0).alias("facing_mean_10"),
        pl.col("curvature_m1").rolling_mean(10).fill_null(0).alias("curvature_m1_mean_10"),
        
        # [NEW] Jitter / Tremor (Standard Deviation)
        # Feature này cực mạnh để bắt hành vi Grooming (đứng yên nhưng rung)
        pl.col("velocity_m1").rolling_std(10).fill_null(0).alias("vel_m1_std_10"),
        pl.col("velocity_m2").rolling_std(10).fill_null(0).alias("vel_m2_std_10"),
    ]
    
    df_pl = df_pl.with_columns(roll_exprs)
    return df_pl

def standardize_tracking_data(df_wide, mouse_ids):
    for m_id in mouse_ids:
        prefix = f"mouse{m_id}_"
        center_x = f"{prefix}body_center_x"; center_y = f"{prefix}body_center_y"
        if center_x not in df_wide.columns:
            x_cols = [c for c in df_wide.columns if c.startswith(prefix) and c.endswith('_x')]
            y_cols = [c for c in df_wide.columns if c.startswith(prefix) and c.endswith('_y')]
            if x_cols:
                df_wide[center_x] = df_wide[x_cols].mean(axis=1)
                df_wide[center_y] = df_wide[y_cols].mean(axis=1)
            else:
                df_wide[center_x] = 0.0; df_wide[center_y] = 0.0
        for part in ['nose', 'tail_base']:
            px = f"{prefix}{part}_x"; py = f"{prefix}{part}_y"
            if px not in df_wide.columns:
                df_wide[px] = df_wide[center_x]; df_wide[py] = df_wide[center_y]
    return df_wide

# --- MAIN INFERENCE LOOP ---
def run_global_inference():
    print("🚀 Starting Global Inference (Platinum Features)...")
    
    # 1. LOAD MODELS
    models = []
    
    print(f"📂 Searching models in: {MODEL_PATH}")
    
    # A. Load XGBoost (Lấy tất cả file .pkl ngoại trừ metadata)
    xgb_files = glob.glob(f'{MODEL_PATH}/*.pkl')
    # Lọc bỏ các file không phải model
    xgb_files = [f for f in xgb_files if 'label_encoder' not in f and 'threshold' not in f and 'feature_names' not in f]
    
    if xgb_files:
        print(f"   🔍 Found {len(xgb_files)} XGBoost models.")
        for f in xgb_files:
            try:
                model = joblib.load(f)
                models.append(('xgb', model))
                print(f"     Loaded: {os.path.basename(f)}")
            except:
                print(f"     ❌ Failed to load: {os.path.basename(f)}")
    else:
        print("   ⚠️ No XGBoost models found.")

    # B. Load CatBoost (Lấy tất cả file .cbm)
    cat_files = glob.glob(f'{MODEL_PATH}/*.cbm')
    
    if cat_files:
        print(f"   🔍 Found {len(cat_files)} CatBoost models.")
        for f in cat_files:
            try:
                clf = CatBoostClassifier()
                clf.load_model(f)
                models.append(('cat', clf))
                print(f"     Loaded: {os.path.basename(f)}")
            except:
                print(f"     ❌ Failed to load: {os.path.basename(f)}")
    else:
        print("   ⚠️ No CatBoost models found.")
            
    if not models:
        print("❌ CRITICAL: Không load được bất kỳ model nào!")
        return
        
    print(f"✅ TOTAL MODELS LOADED: {len(models)}")

    # Load Encoder & Thresholds
    try:
        le = joblib.load(f'{MODEL_PATH}/label_encoder.pkl')
        # [FIX] Load thêm best_thresholds vào biến best_ths
        best_ths = joblib.load(f'{MODEL_PATH}/best_thresholds.pkl') 
        print(f"✅ Configs loaded. Thresholds: {len(best_ths)} classes")
    except Exception as e:
        print(f"❌ CRITICAL ERROR: Không load được configs! Lỗi: {e}")
        return

    # [FIX] Tìm index của class 'other' để dùng cho logic safe-check sau này
    try:
        other_idx = le.transform(['other'])[0]
    except:
        other_idx = -1

    # 2. INIT SUBMISSION
    if os.path.exists(SUBMISSION_FILE): os.remove(SUBMISSION_FILE)
    pd.DataFrame(columns=['row_id', 'video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame']).to_csv(SUBMISSION_FILE, index=False)
    
    test_meta = pd.read_csv(f'{TEST_DATA_PATH}/test.csv')
    row_id_counter = 0
    
    # 3. PROCESS VIDEOS
    for idx, row in tqdm(test_meta.iterrows(), total=len(test_meta), desc="Inferencing"):
        vid = row['video_id']; lab = row['lab_id']
        pix_per_cm = row['pix_per_cm_approx'] if row['pix_per_cm_approx'] > 0 else 1.0
        
        t_path = f'{TEST_DATA_PATH}/test_tracking/{lab}/{vid}.parquet'
        if not os.path.exists(t_path): continue
        
        try:
            # ETL
            track_df = pd.read_parquet(t_path)
            track_df['col_name'] = 'mouse' + track_df['mouse_id'].astype(str) + '_' + track_df['bodypart']
            px = track_df.pivot(index='video_frame', columns='col_name', values='x')
            py = track_df.pivot(index='video_frame', columns='col_name', values='y')
            px.columns = [c + '_x' for c in px.columns]; py.columns = [c + '_y' for c in py.columns]
            df_wide = pd.concat([px, py], axis=1).sort_index()
            
            mouse_ids = sorted(list(set([int(c.split('_')[0].replace('mouse', '')) for c in px.columns if 'mouse' in c])))
            df_wide = standardize_tracking_data(df_wide, mouse_ids)
            
            # [FIX] Giảm limit về 5 để khớp với lúc Train, tránh tạo dữ liệu giả
            df_wide = df_wide.interpolate(limit=5) 
            # [FIX] Fill tuần tự an toàn hơn
            df_wide = df_wide.fillna(method='ffill').fillna(method='bfill').fillna(0)
            
            df_wide = df_wide / pix_per_cm
            df_wide = df_wide.reset_index().rename(columns={'video_frame': 'frame'})
            
            pl_wide = pl.from_pandas(df_wide)
            video_submissions = []
            
            import itertools
            for m1, m2 in itertools.permutations(mouse_ids, 2):
                col_map = {}
                for c in pl_wide.columns:
                    if f'mouse{m1}_' in c: col_map[c] = c.replace(f'mouse{m1}_', 'mouse1_')
                    elif f'mouse{m2}_' in c: col_map[c] = c.replace(f'mouse{m2}_', 'mouse2_')
                
                pair_pl = pl_wide.select(['frame'] + list(col_map.keys())).rename(col_map)
                pair_pl = calculate_features_polars(pair_pl)
                X_test = pair_pl.select(FEATURE_COLS).to_pandas().astype(np.float32)
                
                # PREDICT (Weighted Soft Voting)
                # Trọng số: 0.5 XGB + 0.5 Cat (Cân bằng)
                # Nếu load nhiều folds, chia đều
                final_probs = np.zeros((len(X_test), len(le.classes_)))
                weight_per_model = 1.0 / len(models)
                
                for name, m in models:
                    probs = m.predict_proba(X_test)
                    final_probs += probs * weight_per_model
                
                # [FIX] Logic Safe-Thresholding: Check trực tiếp thay vì chia
                y_pred_idx = np.argmax(final_probs, axis=1)
                
                # Force về 'other' nếu xác suất max không vượt qua threshold riêng của class đó
                for i in range(len(final_probs)):
                    pred_class = y_pred_idx[i]
                    
                    # Nếu dự đoán là hành vi (không phải other)
                    if pred_class != other_idx:
                        # Lấy threshold của hành vi đó (default 0.3 nếu không tìm thấy)
                        th = best_ths.get(pred_class, 0.3) 
                        
                        # Nếu xác suất không đủ tự tin -> Gán về 'other'
                        if final_probs[i, pred_class] < th:
                            y_pred_idx[i] = other_idx
                
                # POST-PROCESS (Min Duration Filter)
                frames = pair_pl['frame'].to_numpy()
                current_action = None; start_f = 0
                
                for i, pred_idx in enumerate(y_pred_idx):
                    action = le.classes_[pred_idx]
                    if action != current_action:
                        if current_action is not None and current_action != 'other':
                            if (i - start_f) >= 2: # Lọc nhiễu < 2 frames
                                video_submissions.append({
                                    'row_id': -1, 'video_id': vid,
                                    'agent_id': f'mouse{m1}', 'target_id': f'mouse{m2}',
                                    'action': current_action, 'start_frame': frames[start_f], 'stop_frame': frames[i-1]
                                })
                        current_action = action; start_f = i
                
                if current_action is not None and current_action != 'other':
                     if (len(y_pred_idx) - start_f) >= 2:
                        video_submissions.append({
                            'row_id': -1, 'video_id': vid,
                            'agent_id': f'mouse{m1}', 'target_id': f'mouse{m2}',
                            'action': current_action, 'start_frame': frames[start_f], 'stop_frame': frames[-1]
                        })

            if video_submissions:
                batch_df = pd.DataFrame(video_submissions)
                batch_df['row_id'] = range(row_id_counter, row_id_counter + len(batch_df))
                row_id_counter += len(batch_df)
                batch_df.to_csv(SUBMISSION_FILE, mode='a', header=False, index=False)
            
            del track_df, df_wide, pl_wide, X_test, final_probs
            gc.collect()

        except Exception as e:
            print(f"⚠️ Err {vid}: {e}")
            continue

    print(f"✅ DONE! Generated {row_id_counter} predictions.")

# Chạy
run_global_inference()

✅ Đường dẫn model OK: /kaggle/input/model-testdata/models
   Tìm thấy 5 file .pkl
🚀 Starting Global Inference (Platinum Features)...
📂 Searching models in: /kaggle/input/model-testdata/models
   🔍 Found 3 XGBoost models.
     Loaded: xgb_seed42.pkl
     Loaded: xgb_seed2023.pkl
     Loaded: xgb_seed9999.pkl
   🔍 Found 3 CatBoost models.
     Loaded: cat_seed2023.cbm
     Loaded: cat_seed9999.cbm
     Loaded: cat_seed42.cbm
✅ TOTAL MODELS LOADED: 6
✅ Configs loaded. Thresholds: 27 classes


Inferencing:   0%|          | 0/1 [00:00<?, ?it/s]

✅ DONE! Generated 1201 predictions.
